In [33]:
# Install needed libraries
#!pip install pandas plotnine

In [22]:
''' 
first I import all modules I will need
'''
import numpy as np
import random
import pandas as pd
from collections import Counter
import pickle

In [23]:
alanine       = {"GCU", "GCC", "GCA", "GCG"}
arginine      = {"CGU", "CGC", "CGA", "CGG", "AGA", "AGG"}
asparagine    = {"AAU", "AAC"}
aspartic_acid = {"GAU", "GAC"}
cysteine      = {"UGU", "UGC"}
glutamine     = {"CAA", "CAG"}
glutamic_acid = {"GAA", "GAG"}
glycine       = {"GGU", "GGC", "GGA", "GGG"}
histidine     = {"CAU", "CAC"}
isoleucine    = {"AUU", "AUC", "AUA"}
leucine       = {'CUG', 'CUA', 'CUU', 'CUC', 'UUA', 'UUG'}
lysine        = {"AAA", "AAG"}
methionine    = {"AUG"}
phenylalanine = {"UUU", "UUC"}
proline       = {"CCU", "CCC", "CCA", "CCG"}
serine        = {"UCU", "UCC", "UCA", "UCG", "AGU", "AGC"}
threonine     = {"ACU", "ACC", "ACA", "ACG"}
tryptophan    = {"UGG"}
tyrosine      = {"UAU", "UAC"}
valine        = {"GUU", "GUC", "GUA", "GUG"}
stop_codons   = {"UAA", "UAG", "UGA"}

amino_acids = {
    "A": alanine,
    "R": arginine,
    "N": asparagine,
    "D": aspartic_acid,
    "C": cysteine,
    "Q": glutamine,
    "E": glutamic_acid,
    "G": glycine,
    "H": histidine,
    "I": isoleucine,
    "L": leucine,
    "K": lysine,
    "M": methionine,
    "F": phenylalanine,
    "P": proline,
    "S": serine,
    "T": threonine,
    "W": tryptophan,
    "Y": tyrosine,
    "V": valine,
    "STOP": stop_codons 
}

codon_to_amino_acid = {codon: aa for aa, codon_set in amino_acids.items() for codon in codon_set}

In [24]:
# Create population of equal counts for every leucine codon
population = np.tile(['CUG', 'CUA', 'CUU', 'CUC', 'UUA', 'UUG'], 100)

In [25]:
fitness_map = {
    "L": 1.0,   "M": 0.85,  "V": 0.77,  "I": 0.77,  "W": 0.68,  
    "Y": 0.65,  "F": 0.65,  "C": 0.61,  "A": 0.57,  "G": 0.54,  
    "P": 0.54,  "H": 0.52,  "S": 0.50,  "T": 0.50,  "Q": 0.46,  
    "K": 0.44,  "R": 0.44,  "N": 0.41,  "D": 0.35,  "E": 0.35,  
    "STOP": 0.0
}

def translation(codon: str) -> int:
    """
    Function that implements translation
    """
    return codon_to_amino_acid[codon]

def get_AA_fitness(population):
    """
    Fitness function that incorporates translation and is scoring codons based on translation, with respect to leucine. 
    Meaning that codons that encode leucine will get more points than those that don't, and it will depend on how dissimilar they are from leucine.
    """
    fitness_scores = []
    for codon in population:
        fitness_scores.append(fitness_map[translation(codon)])
    
    return np.array(fitness_scores, dtype=float)

In [26]:
def get_uniform_fitness(population):
    '''
    Fitness function where all codons area equal
    Returs fitness scores for given population
    '''
    fitness_scores = np.ones_like(population, dtype=float)
            
    return fitness_scores

In [27]:
def get_favour_leucine_fitness(population):
    '''
    Fitness function where leucine codons are favoured    
    Returs fitness scores for given population
    '''
    fitness_scores = np.array([1 if ind in leucine else 0.25 for ind in population], dtype=float)
            
    return fitness_scores

In [28]:
def roulette_selection(population, fitness_scores):
    """
    Function that implements the roulette-wheel selection, probability for anm individual to be in the next generation depends on its fitness score 
    """
    probabilities = fitness_scores / fitness_scores.sum()
    selected_indices = np.random.choice(population, size=len(population), p=probabilities)
    
    return selected_indices

In [29]:
nucleotides = np.array(["A", "C", "G", "U"])
# to what can nucleotides mutate
mutation_map = {n: nucleotides[nucleotides != n] for n in nucleotides}

In [30]:
def random_simple_mutation(population, mut_per_bit):
    '''
    Basic mutation function without any biases or anything
    :param mut_per_bit: probability of single point mutation
    :return: new population, empty array for statistics, empty dict for statistics
    '''
    new_population = []
    mutation_stats = {codon: 0 for codon in leucine}    # keep track of how many nonsynonymous mutations for each codon

    for i in range(len(population)):  # Loop over individuals
        original_codon = population[i]              # a string, e.g. "CUG"
        original_list = list(original_codon)          # list of nucleotides
        mutated_codon = None

        for j, nucleotide in enumerate(original_list):  # Loop over nucleotide positions
            if random.random() < mut_per_bit:  # Mutation occurs at this nucleotide
                options = [n for n in mutation_map[nucleotide]]
                new_list = original_list.copy()
                new_list[j] = random.choice(options)
                mutated_codon = "".join(new_list)
                new_population.append(mutated_codon)
                break  # Only allow one mutation per codon.

        # If no mutation occurred, append the original codon.
        if mutated_codon is None:
            mutated_codon = original_codon
            new_population.append(mutated_codon)

        '''Statistic about which codon most mutate outside of leucine; SANITY CHECK'''
        if mutated_codon not in leucine and original_codon in leucine:
            mutation_stats[original_codon] += 1
        '''Statistic ends here'''

                
    return np.array(["".join(ind) for ind in new_population]), mutation_stats

In [31]:
def calculate_transition_prob(ratio: float):
    """
    ratio 0.5 means both ts and tv happen equally frequently
    """
    return  ratio / (ratio + 0.5) # transition probability given the ratio

In [32]:
def biased_mutation(population, mut_per_bit, TSTV_ratio):
    """
    Function that during mutation takes into consideration ts/tv ratio
    :param TSTV_ratio: ts/tv ratio
    :return: new population
    """
    purines = ("A", "G")
    pyrimidines = ("C", "U")
    transition_prob = calculate_transition_prob(TSTV_ratio)
    mutation_stats = {codon: 0 for codon in leucine}
    new_population = []
    
    for i in range(len(population)):  # Loop over individuals
        original_codon = population[i]              # a string, e.g. "CUG"
        original_list = list(original_codon)          # list of nucleotides
        mutated_codon = None

        for j, nucleotide in enumerate(original_list):  # Loop over nucleotide positions
            if random.random() < mut_per_bit:  # Mutation occurs at this nucleotide
                if random.random() < transition_prob:
                    options = [n for n in mutation_map[nucleotide] if
                               (n in purines and nucleotide in purines) or
                               (n in pyrimidines and nucleotide in pyrimidines)]
                else:
                    options = [n for n in mutation_map[nucleotide] if
                               (n in purines and nucleotide in pyrimidines) or
                               (n in pyrimidines and nucleotide in purines)]

                # Create a copy and mutate the j-th nucleotide.
                new_list = original_list.copy()
                new_list[j] = random.choice(options)
                mutated_codon = "".join(new_list)
                new_population.append(mutated_codon)
                break  # Only allow one mutation per codon.

        # If no mutation occurred, append the original codon.
        if mutated_codon is None:
            mutated_codon = original_codon
            new_population.append(mutated_codon)

        # Statistic about which codon most mutate outside of leucine
        if mutated_codon not in leucine and original_codon in leucine:
            mutation_stats[original_codon] += 1

    return np.array(new_population), mutation_stats

In [33]:
from plotnine import geom_polygon, geom_text, theme_void, coord_fixed, ggplot, aes, element_rect, theme, element_text, \
    scale_fill_manual


def create_pie_chart(final_population, resolution=50):
    """
    Creates a pie chart without using coord_polar by manually computing wedge polygons.      
    Returns a plotnine ggplot object representing the pie chart.
    """
    
    valid_labels = ['CUG', 'CUA', 'CUU', 'CUC', 'UUA', 'UUG']
    grouped_data = [x if x in valid_labels else "Others" for x in final_population]
    counts = pd.Series(grouped_data).value_counts()
    ordered_categories = [label for label in valid_labels if label in counts.index]
    if "Others" in counts.index:
        ordered_categories.append("Others")
    
    distribution = counts[ordered_categories].tolist()
    
    # Compute angles for each slice
    total = sum(distribution)
    fractions = [value / total for value in distribution]
    angles = [fraction * 2 * np.pi for fraction in fractions]
    
    start_angles = []
    end_angles = []
    current_angle = 0
    for angle in angles:
        start_angles.append(current_angle)
        current_angle += angle
        end_angles.append(current_angle)
    
    palette = {
        "CUG": "#1f77b4",  # Dark blue
        "CUA": "#aec7e8",  # Light blue
        "CUU": "#d62728",  # Dark red
        "CUC": "#ff9896",  # Light red
        "UUA": "#2ca02c",  # Dark green
        "UUG": "#98df8a"   # Light green
    }
    
    # Generate the polygon points for each slice
    poly_data = []
    for i, (start, end, label) in enumerate(zip(start_angles, end_angles, ordered_categories)):
        arc_angles = np.linspace(start, end, resolution)
        for theta in arc_angles:
            x = np.cos(theta)
            y = np.sin(theta)
            poly_data.append({"x": x, "y": y, "label": label})
        # Append the center point to close the wedge.
        poly_data.append({"x": 0, "y": 0, "label": label})
    
    df_poly = pd.DataFrame(poly_data)
    
    text_data = []
    label_radius=0.7
    for start, end, fraction, label in zip(start_angles, end_angles, fractions, ordered_categories):
        mid_angle = (start + end) / 2
        x_label = np.cos(mid_angle) * label_radius
        y_label = np.sin(mid_angle) * label_radius
        percentage = f"{fraction*100:.1f}%"
        text_data.append({"x": x_label, "y": y_label, "label": label, "percentage": percentage})
    
    df_text = pd.DataFrame(text_data)
    
    p = (ggplot(df_poly, aes(x='x', y='y', group='label', fill='label')) +
         geom_polygon() +
         coord_fixed() +
         theme_void() +
         theme(
             panel_background=element_rect(fill='white'),
             plot_background=element_rect(fill='white'),
             plot_title=element_text(size=14),
             legend_title=element_text(size=13),
             legend_text=element_text(size=11),
         ) +
         scale_fill_manual(name="Codons", values=palette) +
         # Add percentage labels.
         geom_text(data=df_text, mapping=aes(x='x', y='y', label='percentage'),
                   color='black', size=10, va='center', ha='center')
    )
    
    return p

In [34]:
def average_the_frequencies(frequencies_df, step):
    """
    Averages the frequencies over blocks of 'step' generations.      
    Returns a new DataFrame with averaged frequencies over each block.
    """
    groups = np.arange(len(frequencies_df)) // step
    # Group by labels and compute the mean for each group, then reset index.
    return frequencies_df.groupby(groups).mean().reset_index(drop=True)

In [35]:
from plotnine import geom_line, labs, scale_color_manual, theme_minimal


def plot_frequencies(frequencies_df, step):
    """
    Plots the distribution of codon frequencies over generations using plotnine.      
    Returns a plotnine ggplot object of codon frequencies.
    """
    
    averages_df = average_the_frequencies(frequencies_df, step)
    averages_df.loc[0] = 100
    
    n_bins = len(averages_df)
    averages_df["Generation"] = np.arange(n_bins) * step + step/2
    long_df = averages_df.melt(id_vars=["Generation"], var_name="Codon", value_name="Frequency")
    order = ["CUG", "CUA", "CUU", "CUC", "UUA", "UUG"]
    long_df["Codon"] = pd.Categorical(long_df["Codon"], categories=order, ordered=True)

    palette = {
        "CUG": "#1f77b4",  # Dark blue
        "CUA": "#aec7e8",  # Light blue
        "CUU": "#d62728",  # Dark red
        "CUC": "#ff9896",  # Light red
        "UUA": "#2ca02c",  # Dark green
        "UUG": "#98df8a"   # Light green
    }
    
    # Build the plot using plotnine.
    p = (ggplot(long_df, aes(x="Generation", y="Frequency", color="Codon")) +
         geom_line(size=1) +
         labs(
             x="Generations",
             y="Codon Frequency"
         ) +
         scale_color_manual(values=palette) +
         theme_minimal() +
         theme(
             plot_title=element_text(size=14, weight='bold'),
             axis_title=element_text(size=11),
             axis_title_x=element_text(size=15),
             axis_title_y=element_text(size=15),
             axis_text_x=element_text(size=13),
             axis_text_y=element_text(size=13),
             legend_title=element_text(size=14),
             legend_text=element_text(size=12),
             panel_background=element_rect(fill="white"),
             plot_background=element_rect(fill="white")
         ))
    
    return p

In [36]:
from plotnine import geom_bar


def plot_mutation_bar_plot(frequencies_df, mutation_stats, mut_rate, generations, pop_size=600):
    """
    Histogram of how many times respective codons mutated out of leucine, normalized.
    :param  mutation_stats: nonsynonymous mutations counts for each codon
    Returns a plotnine ggplot object representing bar plot of mutations count.
    """
    palette = {
        "CUG": "#1f77b4",  # Dark blue
        "CUA": "#aec7e8",  # Light blue
        "CUU": "#d62728",  # Dark red
        "CUC": "#ff9896",  # Light red
        "UUA": "#2ca02c",  # Dark green
        "UUG": "#98df8a"   # Light green
    }
    codons = list(mutation_stats.keys())
    # Calculate the mean frequency for each codon.
    mean_freq = np.mean(frequencies_df, axis=0)
    
    # Normalize the mutation counts.
    norm_factor = pop_size * generations * mut_rate
    normalized_counts = {codon: mutation_stats[codon] / norm_factor for codon in codons}
    # Adjust counts by the mean frequency relative to the pop_size.
    final_counts = {codon: normalized_counts[codon] / (mean_freq[codon] / pop_size) for codon in codons}

    df = pd.DataFrame({
        "Codon": codons,
        "MutationCount": [final_counts[c] for c in codons]
    })

    order = ["CUG", "CUA", "CUU", "CUC", "UUA", "UUG"]
    df["Codon"] = pd.Categorical(df["Codon"], categories=order, ordered=True)

    p = (ggplot(df, aes(x="Codon", y="MutationCount", fill="Codon"))
         + geom_bar(stat="identity")
         + labs(
             x="Codon",
             y="Non-synonymous Mutation Count"
         )
         + scale_fill_manual(values=palette)
         + theme_minimal()
         + theme(
             panel_background=element_rect(fill="white"),
             plot_background=element_rect(fill="white"),
             plot_title=element_text(size=14),
             axis_title_x=element_text(size=14),
             axis_title_y=element_text(size=14),
             axis_text_x=element_text(size=13),
             axis_text_y=element_text(size=12),
             legend_title=element_text(size=14),
             legend_text=element_text(size=12),
         ))
    
    return p

In [37]:
def runEvolution(population, generations, fitness_func, mut_per_bit, mutation_func, *args):
    mutation_stats = {codon: 0 for codon in leucine}
    frequencies = pd.DataFrame(0, index=range(generations), columns=list(leucine))
    frequencies.loc[0] = 100
    for i in range(generations):
        fitness_scores = fitness_func(population)
        mutation_pool = roulette_selection(population, fitness_scores)
        population, mutations= mutation_func(mutation_pool, mut_per_bit, *args)
        # update codon mutation statistics
        for codon in mutation_stats.keys():
            mutation_stats[codon] += mutations.get(codon, 0)

        for codon in leucine:
            frequencies.loc[i, codon] = np.count_nonzero(population == codon)

    return population, frequencies, mutation_stats

In [38]:
def run_multiple_simulations(population, generations, mut_per_bit, times, mutation_func, fitness_func, *args):
    codon_counts = {codon: 0 for codon in leucine}
    codon_proportions = []
    frequencies_df_list = []
    codon_stats_list = []

    for run in range(times):
        print(f"Running simulation {run + 1}/{times}...")
        new_population, frequencies_df, codon_stats = runEvolution(
            population, generations, fitness_func, mut_per_bit, mutation_func, *args
        )
        run_codon_count = Counter(new_population)
        # Update the codon_counts dictionary
        for codon in leucine:
            codon_counts[codon] += run_codon_count.get(codon, 0)
        # Count the frequency of each leucine codon in the final population
        codon_proportions.append(np.array([run_codon_count.get(codon, 0) for codon in leucine]))
        frequencies_df_list.append(frequencies_df)
        codon_stats_list.append(codon_stats)

    # Compute average frequencies over runs.
    avg_frequencies = pd.concat(frequencies_df_list).groupby(level=0).mean()

    # Compute average codon stats for each leucine codon over runs.
    avg_codon_stats = {}
    for codon in leucine:
        avg_codon_stats[codon] = np.mean([cs[codon] for cs in codon_stats_list])

    flat_population = []
    for codon, count in codon_counts.items():
        flat_population.extend([codon] * count)

    return avg_frequencies, avg_codon_stats, codon_proportions, flat_population

<h1>Simple Model</h1>

In [45]:
random.seed(42)
np.random.seed(42)
# Tha simulation was run 5,000 times with 4,000 generations in each
generations = 5
times = 5

In [46]:
mut_per_bit = 0.005

avg_frequencies_SM, avg_codon_stats_SM, codon_proportions_SM, population_SM = run_multiple_simulations(population, generations, mut_per_bit, times, random_simple_mutation, get_favour_leucine_fitness)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [47]:
"""
# Save the inputs using pickle
with open("simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_SM, avg_codon_stats_SM, codon_proportions_SM, population_SM), f)

# Create Plots
pie = create_pie_chart(population_SM)
pie.save("Simple_model/pie_chart.png", width=6, height=4, dpi=300)

freq = plot_frequencies(avg_frequencies_SM, generations // 50)
freq.save("Simple_model/freq_chart.png", width=6, height=4, dpi=300)

mut = plot_mutation_bar_plot(avg_frequencies_SM, avg_codon_stats_SM, mut_per_bit, generations, pop_size=600)
mut.save("Simple_model/mutation.png", width=6, height=4, dpi=300)
"""

'\n# Save the inputs using pickle\nwith open("simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_SM, avg_codon_stats_SM, codon_proportions_SM, population_SM), f)\n\n# Create Plots\npie = create_pie_chart(population_SM)\npie.save("Simple_model/pie_chart.png", width=6, height=4, dpi=300)\n\nfreq = plot_frequencies(avg_frequencies_SM, generations // 50)\nfreq.save("Simple_model/freq_chart.png", width=6, height=4, dpi=300)\n\nmut = plot_mutation_bar_plot(avg_frequencies_SM, avg_codon_stats_SM, mut_per_bit, generations, pop_size=600)\nmut.save("Simple_model/mutation.png", width=6, height=4, dpi=300)\n'

<h1>Mutation Bias Model</h1>

<h2>TS/TV ratio of 1</h2>

In [50]:
ratio = 1.0
avg_frequencies_MBM_1, avg_codon_stats_MBM_1, codon_proportions_MBM_1, population_MBM_1 = run_multiple_simulations(population, generations, mut_per_bit, times, biased_mutation, get_favour_leucine_fitness, ratio)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [51]:
"""
# Save the inputs using pickle
with open("TS_TV_Model/ts_tv_1/simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_MBM_1, avg_codon_stats_MBM_1, codon_proportions_MBM_1, population_MBM_1), f)

pie3 = create_pie_chart(population_MBM_1)
pie3.save("TS_TV_Model/ts_tv_1/pie_chart.png", width=6, height=4, dpi=300)

freq3 = plot_frequencies(avg_frequencies_MBM_1, generations // 50)
freq3.save("TS_TV_Model/ts_tv_1/freq_chart.png", width=6, height=4, dpi=300)

mut3 = plot_mutation_bar_plot(avg_frequencies_MBM_1, avg_codon_stats_MBM_1, mut_per_bit, generations, pop_size=600)
mut3.save("TS_TV_Model/ts_tv_1/mutation.png", width=6, height=4, dpi=300)
"""

'\n# Save the inputs using pickle\nwith open("TS_TV_Model/ts_tv_1/simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_MBM_1, avg_codon_stats_MBM_1, codon_proportions_MBM_1, population_MBM_1), f)\n\npie3 = create_pie_chart(population_MBM_1)\npie3.save("TS_TV_Model/ts_tv_1/pie_chart.png", width=6, height=4, dpi=300)\n\nfreq3 = plot_frequencies(avg_frequencies_MBM_1, generations // 50)\nfreq3.save("TS_TV_Model/ts_tv_1/freq_chart.png", width=6, height=4, dpi=300)\n\nmut3 = plot_mutation_bar_plot(avg_frequencies_MBM_1, avg_codon_stats_MBM_1, mut_per_bit, generations, pop_size=600)\nmut3.save("TS_TV_Model/ts_tv_1/mutation.png", width=6, height=4, dpi=300)\n'

<h2>TS/TV ratio of 2</h2>

In [52]:
ratio = 2.0
avg_frequencies_MBM_2, avg_codon_stats_MBM_2, codon_proportions_MBM_2, population_MBM_2 = run_multiple_simulations(population, generations, mut_per_bit, times, biased_mutation, get_favour_leucine_fitness, ratio)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [53]:
"""
# Save the inputs using pickle
with open("TS_TV_Model/ts_tv_2/simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_MBM_2, avg_codon_stats_MBM_2, codon_proportions_MBM_2, population_MBM_2), f)

pie2 = create_pie_chart(population_MBM_2)
pie2.save("TS_TV_Model/ts_tv_2/pie_chart.png", width=6, height=4, dpi=300)

freq2 = plot_frequencies(avg_frequencies_MBM_2, generations // 50)
freq2.save("TS_TV_Model/ts_tv_2/freq_chart.png", width=6, height=4, dpi=300)

mut2 = plot_mutation_bar_plot(avg_frequencies_MBM_2, avg_codon_stats_MBM_2, mut_per_bit, generations, pop_size=600)
mut2.save("TS_TV_Model/ts_tv_2/mutation.png", width=6, height=4, dpi=300)
"""

'\n# Save the inputs using pickle\nwith open("TS_TV_Model/ts_tv_2/simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_MBM_2, avg_codon_stats_MBM_2, codon_proportions_MBM_2, population_MBM_2), f)\n\npie2 = create_pie_chart(population_MBM_2)\npie2.save("TS_TV_Model/ts_tv_2/pie_chart.png", width=6, height=4, dpi=300)\n\nfreq2 = plot_frequencies(avg_frequencies_MBM_2, generations // 50)\nfreq2.save("TS_TV_Model/ts_tv_2/freq_chart.png", width=6, height=4, dpi=300)\n\nmut2 = plot_mutation_bar_plot(avg_frequencies_MBM_2, avg_codon_stats_MBM_2, mut_per_bit, generations, pop_size=600)\nmut2.save("TS_TV_Model/ts_tv_2/mutation.png", width=6, height=4, dpi=300)\n'

<h2>TS/TV ratio of 5</h2>

In [54]:
ratio = 5.0
avg_frequencies_MBM_5, avg_codon_stats_MBM_5, codon_proportions_MBM_5, population_MBM_5 = run_multiple_simulations(population, generations, mut_per_bit, times, biased_mutation, get_favour_leucine_fitness, ratio)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [55]:
"""
# Save the inputs using pickle
with open("TS_TV_Model/ts_tv_5/simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_MBM_5, avg_codon_stats_MBM_5, codon_proportions_MBM_5, population_MBM_5), f)

pie5 = create_pie_chart(population_MBM_5)
pie5.save("TS_TV_Model/ts_tv_5/pie_chart.png", width=6, height=4,
          dpi=300)

freq5 = plot_frequencies(avg_frequencies_MBM_5, generations // 50)
freq5.save("TS_TV_Model/ts_tv_5/freq_chart.png", width=6, height=4,
           dpi=300)

mut5 = plot_mutation_bar_plot(avg_frequencies_MBM_5, avg_codon_stats_MBM_5, mut_per_bit, generations, pop_size=600)
mut5.save("TS_TV_Model/ts_tv_5/mutation.png", width=6, height=4, dpi=300)
          """

'\n# Save the inputs using pickle\nwith open("TS_TV_Model/ts_tv_5/simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_MBM_5, avg_codon_stats_MBM_5, codon_proportions_MBM_5, population_MBM_5), f)\n\npie5 = create_pie_chart(population_MBM_5)\npie5.save("TS_TV_Model/ts_tv_5/pie_chart.png", width=6, height=4,\n          dpi=300)\n\nfreq5 = plot_frequencies(avg_frequencies_MBM_5, generations // 50)\nfreq5.save("TS_TV_Model/ts_tv_5/freq_chart.png", width=6, height=4,\n           dpi=300)\n\nmut5 = plot_mutation_bar_plot(avg_frequencies_MBM_5, avg_codon_stats_MBM_5, mut_per_bit, generations, pop_size=600)\nmut5.save("TS_TV_Model/ts_tv_5/mutation.png", width=6, height=4, dpi=300)\n          '

<h1>Translation Model</h1>

<h2>TS/TV ratio of 0.5</h2>

In [56]:
avg_frequencies_TM_0_5, avg_codon_stats_TM_0_5, codon_proportions_TM_0_5, population_TM_0_5 = run_multiple_simulations(population, generations, mut_per_bit, times, random_simple_mutation, get_AA_fitness)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [57]:
"""
# Save the inputs using pickle
with open("Translation_Model/ts_tv_0_5/simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_TM_0_5, avg_codon_stats_TM_0_5, codon_proportions_TM_0_5, population_TM_0_5), f)

pie0_5 = create_pie_chart(population_TM_0_5)
pie0_5.save("Translation_Model/ts_tv_0_5/pie_chart.png", width=6, height=4,
          dpi=300)

freq0_5 = plot_frequencies(avg_frequencies_TM_0_5, generations // 50)
freq0_5.save("Translation_Model/ts_tv_0_5/freq_chart.png", width=6, height=4,
           dpi=300)

mut0_5 = plot_mutation_bar_plot(avg_frequencies_TM_0_5, avg_codon_stats_TM_0_5, mut_per_bit, generations, pop_size=600)
mut0_5.save("Translation_Model/ts_tv_0_5/mutation.png", width=6, height=4, dpi=300)
"""

'\n# Save the inputs using pickle\nwith open("Translation_Model/ts_tv_0_5/simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_TM_0_5, avg_codon_stats_TM_0_5, codon_proportions_TM_0_5, population_TM_0_5), f)\n\npie0_5 = create_pie_chart(population_TM_0_5)\npie0_5.save("Translation_Model/ts_tv_0_5/pie_chart.png", width=6, height=4,\n          dpi=300)\n\nfreq0_5 = plot_frequencies(avg_frequencies_TM_0_5, generations // 50)\nfreq0_5.save("Translation_Model/ts_tv_0_5/freq_chart.png", width=6, height=4,\n           dpi=300)\n\nmut0_5 = plot_mutation_bar_plot(avg_frequencies_TM_0_5, avg_codon_stats_TM_0_5, mut_per_bit, generations, pop_size=600)\nmut0_5.save("Translation_Model/ts_tv_0_5/mutation.png", width=6, height=4, dpi=300)\n'

<h2>TS/TV ratio of 1</h2>

In [58]:
ratio = 1.0
avg_frequencies_TM_1, avg_codon_stats_TM_1, codon_proportions_TM_1, population_TM_1 = run_multiple_simulations(population, generations, mut_per_bit, times, biased_mutation, get_AA_fitness, ratio)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [59]:
"""
# Save the inputs using pickle
with open("Translation_Model/ts_tv_1/simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_TM_1, avg_codon_stats_TM_1, codon_proportions_TM_1, population_TM_1), f)

pie6 = create_pie_chart(population_TM_1)
pie6.save("Translation_Model/ts_tv_1/pie_chart.png", width=6, height=4,
          dpi=300)

freq6 = plot_frequencies(avg_frequencies_TM_1, generations // 50)
freq6.save("Translation_Model/ts_tv_1/freq_chart.png", width=6, height=4,
           dpi=300)

mut6 = plot_mutation_bar_plot(avg_frequencies_TM_1, avg_codon_stats_TM_1, mut_per_bit, generations, pop_size=600)
mut6.save("Translation_Model/ts_tv_1/mutation.png", width=6, height=4, dpi=300)
"""

'\n# Save the inputs using pickle\nwith open("Translation_Model/ts_tv_1/simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_TM_1, avg_codon_stats_TM_1, codon_proportions_TM_1, population_TM_1), f)\n\npie6 = create_pie_chart(population_TM_1)\npie6.save("Translation_Model/ts_tv_1/pie_chart.png", width=6, height=4,\n          dpi=300)\n\nfreq6 = plot_frequencies(avg_frequencies_TM_1, generations // 50)\nfreq6.save("Translation_Model/ts_tv_1/freq_chart.png", width=6, height=4,\n           dpi=300)\n\nmut6 = plot_mutation_bar_plot(avg_frequencies_TM_1, avg_codon_stats_TM_1, mut_per_bit, generations, pop_size=600)\nmut6.save("Translation_Model/ts_tv_1/mutation.png", width=6, height=4, dpi=300)\n'

<h2>TS/TV ratio of 2</h2>

In [60]:
ratio = 2.0
avg_frequencies_TM_2, avg_codon_stats_TM_2, codon_proportions_TM_2, population_TM_2 = run_multiple_simulations(population, generations, mut_per_bit, times, biased_mutation, get_AA_fitness, ratio)

Running simulation 1/5...
Running simulation 2/5...
Running simulation 3/5...
Running simulation 4/5...
Running simulation 5/5...


In [61]:
"""
# Save the inputs using pickle
with open("Translation_Model/ts_tv_2/simulation_result.pkl", "wb") as f:
    pickle.dump((avg_frequencies_TM_2, avg_codon_stats_TM_2, codon_proportions_TM_2, population_TM_2), f)

pie7 = create_pie_chart(population_TM_2)
pie7.save("Translation_Model/ts_tv_2/pie_chart.png", width=6, height=4,
          dpi=300)

freq7 = plot_frequencies(avg_frequencies_TM_2, generations // 50)
freq7.save("Translation_Model/ts_tv_2/freq_chart.png", width=6, height=4,
           dpi=300)

mut7 = plot_mutation_bar_plot(avg_frequencies_TM_2, avg_codon_stats_TM_2, mut_per_bit, generations, pop_size=600)
mut7.save("Translation_Model/ts_tv_2/mutation.png", width=6, height=4, dpi=300)
"""

'\n# Save the inputs using pickle\nwith open("Translation_Model/ts_tv_2/simulation_result.pkl", "wb") as f:\n    pickle.dump((avg_frequencies_TM_2, avg_codon_stats_TM_2, codon_proportions_TM_2, population_TM_2), f)\n\npie7 = create_pie_chart(population_TM_2)\npie7.save("Translation_Model/ts_tv_2/pie_chart.png", width=6, height=4,\n          dpi=300)\n\nfreq7 = plot_frequencies(avg_frequencies_TM_2, generations // 50)\nfreq7.save("Translation_Model/ts_tv_2/freq_chart.png", width=6, height=4,\n           dpi=300)\n\nmut7 = plot_mutation_bar_plot(avg_frequencies_TM_2, avg_codon_stats_TM_2, mut_per_bit, generations, pop_size=600)\nmut7.save("Translation_Model/ts_tv_2/mutation.png", width=6, height=4, dpi=300)\n'